In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [2]:
test_inst = 'px{a<2006:qkq,m>2090:A,rfg}'
test_part = '{x=787,m=2655,a=1222,s=2876}'

In [9]:
def process_part(part_str):
    part_str =part_str.replace('=',':')
    for l in list('xmas'):
        part_str = part_str.replace(l,f"'{l}'")
    part_dict = eval(part_str)
    return part_dict

In [87]:
class Rule:
    def work_part(self,part_dict):
        x=part_dict['x']
        m=part_dict['m']
        a=part_dict['a']
        s=part_dict['s']
        
        for rule in self.rules:
            if ':' not in rule:
                return rule
            
            condition, dest = rule.split(':')
            if eval(condition) :
                return dest
    
    
    def __init__(self, process):
        name, process = process.split('{')
        process = process.replace('}','')
        self.name = name
        self.rules = process.split(',')
        
        conditions =[]
        destinations =[]
        
        for rule in self.rules:
            if ':' not in rule:
                conditions.append('')
                destinations.append(rule)
            else:
                condition, dest = rule.split(':')
                conditions.append(condition)
                destinations.append(dest)
        self.conditions = conditions
        self.destinations = destinations
        
        
        

In [88]:
test_work = Rule(test_inst)

In [89]:
test_work.work_part(process_part(test_part))

'qkq'

In [90]:
def do_part_one(file_path):
    work_flows = {}
    parts = []
    
    with open(file_path,'r') as f:
        process_rules=True
        for line in f.readlines():
            line = line.strip()
            if len(line) < 1:
                process_rules = False
                continue
                
            if process_rules:
                r = Rule(line)
                work_flows[r.name] = r
            else:
                parts.append(process_part(line))
    
    total = 0 
    for part in parts:
        dest = 'in'
        while dest not in [ 'R', 'A']:
            dest = work_flows[dest].work_part(part)
        if dest == 'A':
            total += sum(part.values())
        
                
    
    return total

In [91]:
%%time
do_part_one('input_data/test_19.txt')

CPU times: user 2.28 ms, sys: 1.85 ms, total: 4.13 ms
Wall time: 2.59 ms


19114

In [92]:
%%time
do_part_one('input_data/day_19.txt')

CPU times: user 31.9 ms, sys: 1.49 ms, total: 33.4 ms
Wall time: 33.1 ms


333263

# Part 2

In [145]:

def finding_accepted_paths(workflows):
    paths=[]
    def add_pathes(cur, path_str,paths ):
        if cur =='A':
            paths.append(path_str)
            
        else:
            for dest in workflows[cur].destinations:
                if dest !='R':
                    new_path= path_str+'->'+dest
                    add_pathes(dest, new_path, paths)
                
    
    add_pathes('in', 'in', paths)
    return paths
    

In [159]:
def get_path_possibility(path,workflows):
    bounds = {l: [1,4000] for l in list('xmas')}
    dests = path.split('->')
    for i in range(len(dests)-1):
        cur = dests[i]
        desired = dests[i+1]
        process = workflows[cur]
       
        possible = sum([ d==desired for d in process.destinations])
        
        for cond, dest in zip(process.conditions,process.destinations):
            if dest == desired:
                if '<' in cond: 
                    letter, val = cond.split('<')
                    if bounds[letter][1] >= int(val):
                        bounds[letter][1] = int(val)-1
                if '>' in cond: 
                    letter, val = cond.split('>')
                    if bounds[letter][0] <= int(val):
                        bounds[letter][0] = int(val)+1
                break
                
            else:
                if '<' in cond: 
                    letter, val = cond.split('<')
                    if bounds[letter][0] < int(val):
                        bounds[letter][0] = int(val)
                if '>' in cond: 
                    letter, val = cond.split('>')
                    if bounds[letter][1] > int(val):
                        bounds[letter][1] = int(val)
                    
    total=1
    for rang in bounds.values():
        if rang[1] < rang[0] :
            return 0 
        total*=(rang[1] - rang[0])+1
        
    print(bounds, total)
    return  total

In [182]:
from copy import deepcopy

def finding_accepted_paths(workflows):
    def find_number_combo_paths(cur, bounds):
        if cur =='A':
            number_to=1
            for rang in bounds.values():
                if rang[1] < rang[0] :
                    number_to = 0 
                    break
                number_to*=(rang[1] - rang[0])+1
            return number_to
        if cur == 'R':
            return 0
            
        elif cur != 'R':
            process = workflows[cur]
            totals = []
            for cond, dest in zip(process.conditions,process.destinations):
                stop_bounds = deepcopy(bounds)
                if '<' in cond: 
                    letter, val = cond.split('<')
                    if stop_bounds[letter][1] >= int(val):
                        stop_bounds[letter][1] = int(val)-1
                        
                    if bounds[letter][0] < int(val):
                        bounds[letter][0] = int(val)
                        
                        
                if '>' in cond: 
                    letter, val = cond.split('>')
                    if stop_bounds[letter][0] <= int(val):
                        stop_bounds[letter][0] = int(val)+1
                        
                    if bounds[letter][1] > int(val):
                        bounds[letter][1] = int(val)
                        
                totals.append(find_number_combo_paths(dest,stop_bounds))
            return sum(totals)
                
                
    bounds = {l: [1,4000] for l in list('xmas')}
    total =0
    total = find_number_combo_paths('in', bounds)
    return total
    

In [183]:
def do_part_two(file_path):
    work_flows = {}
    
    with open(file_path,'r') as f:
        
        for line in f.readlines():
            line = line.strip()
            if len(line) < 1:
                break
                
            r = Rule(line)
            work_flows[r.name] = r
                
    return finding_accepted_paths(work_flows)
    paths = set(finding_accepted_paths(work_flows))
    paths = list(paths)
    total=0 
    for p in paths:
        print(p)
        total+=get_path_possibility(p, work_flows)
        
    return total

In [185]:
%%time
test = do_part_two('input_data/test_19.txt')
print (test == 167409079868000,test)


True 167409079868000
CPU times: user 2.31 ms, sys: 1.78 ms, total: 4.1 ms
Wall time: 2.7 ms


In [187]:
%%time
do_part_two('input_data/day_19.txt')

           

CPU times: user 25.9 ms, sys: 2.06 ms, total: 28 ms
Wall time: 26.7 ms


130745440937650